<a href="https://colab.research.google.com/github/jvilchesf/Portfolio/blob/main/QC_Dashboard_qc_list.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import auth
from google.auth import default
from datetime import datetime

import numpy as np
import pandas as pd
import gspread
from gspread_dataframe import set_with_dataframe

auth.authenticate_user()
creds, _ = default()

gc = gspread.authorize(creds)
sh_master = gc.open('2024_Breeding Season_QC List  - VH (5)') #Read gsheet file
sh_schedule = gc.open('Breeding Season 2024 Schedule') #Read gsheet file


In [ ]:
# Function to handle merged cells for the first three columns
def fill_merged_cells(data):
    for row_index, row in enumerate(data):
        for col_index in range(3):  # Limit to the first three columns
            cell = row[col_index]
            if cell == '':
                # Find the closest non-empty cell above
                for k in range(row_index - 1, -1, -1):
                    if data[k][col_index] != '':
                        data[row_index][col_index] = data[k][col_index]
                        break
    return data

In [ ]:
# Step 3: Define the columns to keep
columns_to_keep = [
    'Project Code',
    'Month',
    'Fieldwork',
    'Further Fieldwork Information',
    'Site Visit Completed By',
    'Scheduled',
    'Data QC\'d',
    'Data Entered in Master Excel',
    'Shapefile Produced (GIS)'
]

# Step 4: Create an empty list to hold DataFrames for each sheet
df_list = []

# Step 5: Loop through sheets 5 to 61
for sheet_index in range(7, 65):  # Sheets are 0-indexed, so sheet 5 is index 4
#for sheet_index in range(5, 63):
    worksheet = sh_master.get_worksheet(sheet_index)
    if worksheet is not None:
        # Get all values in the worksheet as a list of lists
        data = worksheet.get_all_values()
        if data:
            # Handle merged cells
            data = fill_merged_cells(data)
            # Convert the data to a DataFrame
            df = pd.DataFrame(data[2:], columns=data[1])  # Using the second row (index 1) as the header
            # Ensure the DataFrame contains the required columns
            # Select only the specified columns
            df = df.iloc[:, :11]
            # Check for the column with the leading space and rename it
            if ' Project Code' in df.columns:
                df.rename(columns={' Project Code': 'Project Code'}, inplace=True)

            df_list.append(df)


In [ ]:
# Step 6: Concatenate all DataFrames into one
if df_list:
    df_qc_list = pd.concat(df_list, ignore_index=True)
    df_qc_list = df_qc_list.drop(['Date Scheduled',' Staysafe / Whatsapp'],axis=1)
    df_qc_list['execution_date'] = datetime.now().strftime("%Y-%m-%d")

    # Open the existing Google Sheet by its title or URL
    spreadsheet = gc.open('QC_list_summarize_v1')

    # Select the first sheet (or the sheet you want to append to)
    worksheet = spreadsheet.get_worksheet(0)

    # Find the next available row
    existing_data = worksheet.get_all_values()
    next_row = len(existing_data) + 1  # Adding 1 because Google Sheets index starts at 1, not 0

    # Prepare the range for updating
    # Calculate the range to update
    rows, cols = df_qc_list.shape
    cell_range = f'A{next_row}:J{next_row + rows - 1}'  # Adjust columns as needed


    # Convert the DataFrame to a list of lists for updating
    data_to_append = df_qc_list.values.tolist()

    # Update the range of cells with the new data
    cell_list = worksheet.range(cell_range)

    for cell, data in zip(cell_list, sum(data_to_append, [])):
        cell.value = data

    worksheet.update_cells(cell_list, value_input_option='USER_ENTERED')
else:
    print("No valid data found in the specified sheets.")